In [1]:
import pandas as pd
import ast
import numpy as np
import sklearn.metrics.pairwise
import copy

# import product and ingredient table

In [2]:
#import product data table
df_pd = pd.read_csv("product_data.csv")
df_ing = pd.read_csv("ingredient_data.csv")

In [3]:
#convert ingredient and category columns into list
for i in range(len(df_pd)):
    df_pd.at[i,'ing_set']=set(ast.literal_eval(df_pd.loc[i,'ing_set']))
    df_pd.at[i,'cate']=set(ast.literal_eval(df_pd.loc[i,'cate']))

In [4]:
#convert ingredient function columns into list, ewg level into float
for i in range(len(df_ing)):
    df_ing.at[i,'ing_func']=ast.literal_eval(df_ing.loc[i,'ing_func'])
    if str(df_ing.loc[i,'cir'])=='nan':
        df_ing.at[i,'cir']=None
    if str(df_ing.loc[i,'ewg'])=='nan':
        df_ing.at[i,'ewg']=None
    else:
        df_ing.at[i,'ewg']=float(df_ing.loc[i,'ewg'])

# create customer table

In [5]:
#create data frame of customer
df_customer=pd.DataFrame()
#create columns
df_customer['name']=None
df_customer['rmd']=None
df_customer['alg_ind'] = None
df_customer['nalg_ind'] = None
df_customer['alg_ing'] = None
df_customer['nalg_ing'] = None
df_customer['cosine']=None

# create recommend function

In [6]:
#create function that add index to dataframe columns 
def add_ind_product(name,alg,nalg):
    for ind,itm in enumerate(df_customer['name']):
        if itm==name:
            df_customer['alg_ind'][ind]=alg
            df_customer['nalg_ind'][ind]=nalg

In [7]:
#create function that collect each ingredient of product
def add_ing(name):
    alg_set=set()
    nalg_set=set()
    for ind,itm in enumerate(df_customer['name']):
        if itm == name:
            name_ind=ind
    for alg in df_customer['alg_ind'][name_ind]:
        for itm in df_pd['ing_set'][alg]:
            for ind,ing in enumerate(df_ing['ing']):
                if itm==ing:
                    alg_set.add(itm)
    df_customer['alg_ing'][name_ind]=alg_set
    for nalg in df_customer['nalg_ind'][name_ind]:
        for itm in df_pd['ing_set'][nalg]:
            for ind,ing in enumerate(df_ing['ing']):
                if itm==ing:
                    nalg_set.add(itm)
    df_customer['nalg_ing'][name_ind]=nalg_set
    df_customer['alg_ing'][name_ind]=df_customer['alg_ing'][name_ind]-df_customer['nalg_ing'][name_ind]
    
    df_customer['alg_ing'][name_ind]=add_ewg(df_customer['alg_ing'][name_ind])
    df_customer['nalg_ing'][name_ind]=add_ewg(df_customer['nalg_ing'][name_ind])

In [8]:
#create function that collect cir, ewg level, and ingredient function of each ingredient of product
def add_ewg(ingredient):
    ewg_cir=[]
    dict_ing=dict()
    for ing in ingredient:
        for ind,itm in enumerate(df_ing['ing']):
            if itm==ing:
                ewg_cir=[int(df_ing['ewg'][ind]),df_ing['cir'][ind],df_ing['ing_func'][ind]]
                dict_ing.update({ing:ewg_cir})
    return dict_ing

In [9]:
def cosine(name_ctm):
    name=name_ctm

    for ind,itm in enumerate(df_customer['name']):
        if itm == name:
            name_ind=ind 
#find category of alergic product 
    item_type=[]
    for ind_alg in df_customer['alg_ind'][name_ind]:
        item=set()
        for ind_pd,pd in enumerate(df_pd['cate']):
            if ind_alg==ind_pd:
                item_type_temp=copy.deepcopy(pd)
                if 'Face Skincare' in item_type_temp and len(item_type_temp)>1:
                    item_type_temp.remove('Face Skincare')
                for i in item_type_temp:
                    item.add(i)
        item_type.append(item)
#find index of product that not have alergic ingredient and have same type of alergic product               
    list_cosine=[]
    for i in range(len(item_type)):
        ind_pd_rec=[]
        for ty in item_type[i]:
            if ty=='Face Skincare':
                for ind_pd,pd in enumerate(df_pd['cate']):
                    if pd==item_type and bool(df_pd['ing_set'][ind_pd] & set(df_customer['alg_ing'][name_ind].keys()))==False:
                        ind_pd_rec.append(ind_pd)
            else:             
                for ind_pd,pd in enumerate(df_pd['cate']):
                    if ty in pd and bool(df_pd['ing_set'][ind_pd] & set(df_customer['alg_ing'][name_ind].keys()))==False:
                        ind_pd_rec.append(ind_pd)
    
        for ind_alg in df_customer['alg_ind'][name_ind]:
            dict_cosine=dict()
            for ind in ind_pd_rec:
                ing_union=df_pd['ing_set'][ind]|df_pd['ing_set'][ind_alg]
                vec_a=[]
                vec_b=[]
                for ing in ing_union:
                    if ing in df_pd['ing_set'][ind]:
                        vec_a.append(1)
                    else:
                        vec_a.append(0)
                    if ing in df_pd['ing_set'][ind_alg]:
                        vec_b.append(1)
                    else:
                        vec_b.append(0)
                cosine=sklearn.metrics.pairwise. cosine_similarity([vec_a],[vec_b])
                dict_cosine.update({cosine[0][0]:ind})
        list_cosine.append(dict_cosine)
    df_customer['cosine'][name_ind]=list_cosine

In [10]:
def recommend(name):
    add_ing(name)
    cosine(name)

In [11]:
#find index product
for ind,itm in enumerate(df_pd['brand']):
    if "Cathy Doll".lower() in itm.lower() and 'Acne Solution Serum Foam Cleanser'.lower() in df_pd['product'][ind].lower():
        print(ind,':',itm,':',df_pd['product'][ind])

8306 : Cathy Doll : Acne Solution Serum Foam Cleanser


# insert information to table

In [12]:
#add Mean's information to table
df_customer = df_customer.append({'name': 'mean'}, ignore_index=True)
add_ind_product('mean',[20740],[1789,281,197,9153,4402,3018,24653])
recommend('mean')

In [13]:
#add May's information to table 
df_customer = df_customer.append({'name': 'may'}, ignore_index=True)
add_ind_product('may',[3171,18197,3300,12559,22615,3267],[3832,1973,879,15281,13618,10387,1734,8325,62])
recommend('may')

In [14]:
#add Oly's information to table
df_customer = df_customer.append({'name': 'oly'}, ignore_index=True)
add_ind_product('oly',[8306],[580,3593,3308,18067,19519])
recommend('oly')

In [15]:
#add Preaw's information to table
df_customer = df_customer.append({'name': 'preaw-1'}, ignore_index=True)
add_ind_product('preaw-1',[3267],[27910,23525])
recommend('preaw-1')

In [16]:
#add Preaw's information to table
df_customer = df_customer.append({'name': 'preaw-2'}, ignore_index=True)
add_ind_product('preaw-2',[3267],[3333])
recommend('preaw-2')

In [17]:
#add Oly's information to table
df_customer = df_customer.append({'name': 'cinth'}, ignore_index=True)
add_ind_product('cinth',[3028],[3220,4451])
recommend('cinth')

In [18]:
def show_recmd_product(name_ctm):
    name=name_ctm
    for ind,itm in enumerate(df_customer['name']):
        if itm == name:
            name_ind=ind
    dict_rmd=dict()
    for ind,itm in enumerate(df_customer['alg_ind'][name_ind]):
        print('คุณแพ้ผลิตภัณฑ์ :',df_pd['brand'][itm],df_pd['product'][itm])
        print('ประเภท:',str(df_pd['cate'][itm])[1:-1])
        print('เดาว่าคุณแพ้สาร:')
        for jnd,jtm in enumerate(df_customer['alg_ing'][name_ind]):
            print(jtm,end=', ')
            if jnd==2:              
                break
        num=len(df_customer['alg_ing'][name_ind])
        print(f'และสารอื่นๆอีกรวม {num} ชนิด')
        len_ing_ctm=len(df_customer['alg_ing'][name_ind])+len(df_customer['nalg_ing'][name_ind])
        len_ing_all=len(df_ing['ing'])
        preci=len_ing_ctm/len_ing_all
        print(f'ด้วยความแม่นยำ {preci*100:.2f} % (มีข้อมูลสาร {len_ing_ctm} ชนิด, จาก {len_ing_all} ชนิด)')
        print(f'หมายเหตุ: ยิ่งป้อนข้อมูลประวัติการใช้ผลิตภัณฑ์บำรุงผิวของคุณให้มากขึ้น ยิ่งทำให้ความแม่นยำในการคาดเดามากขึ้น')
        
        max_list=copy.deepcopy(df_customer['cosine'][name_ind][ind])
        while True:
            max_cosine=max(max_list)
            ind_max=df_customer['cosine'][name_ind][ind].get(max_cosine)
            if df_customer['cosine'][name_ind][ind].get(max_cosine) not in df_customer.loc[name_ind,'nalg_ind'] and df_pd['brand'][ind_max] !=df_pd['brand'][itm]:
                break
            else:
                del max_list[max_cosine]
        ind_max=df_customer['cosine'][name_ind][ind].get(max_cosine)
        dict_rmd.update({itm:ind_max})
        print('\n!!เราขอแนะนำ!! :',df_pd['brand'][ind_max],df_pd['product'][ind_max])
        print('ประเภท:',str(df_pd['cate'][ind_max])[1:-1])
        print(f'มีความคล้ายผลิตภัณฑ์เดิม {max_cosine*100:.2f} %')
        for i in range(2):
            del max_list[max_cosine]
            max_cosine=max(max_list)
            ind_max=df_customer['cosine'][name_ind][ind].get(max_cosine)
            print(f'!หรืออันดับ! {i+2} : ',df_pd['brand'][ind_max],df_pd['product'][ind_max],'\nประเภท:',str(df_pd['cate'][ind_max])[1:-1])
            print(f'มีความคล้ายผลิตภัณฑ์เดิม {max_cosine*100:.2f} %')
        print('----------------------------------------------------------------\n\n')
    df_customer.at[name_ind,'rmd']=dict_rmd

In [22]:
show_recmd_product('mean')
#show_recmd_product('may')
#show_recmd_product('oly')
#show_recmd_product('preaw-1')
#show_recmd_product('preaw-2')
#show_recmd_product('cinth')

คุณแพ้ผลิตภัณฑ์ : Burt's Bees Replenishing Lip Balm With Pomegranate Oil
ประเภท: 'Lip Balms', 'Face Skincare', 'Lip Care'
เดาว่าคุณแพ้สาร:
Eugenol, Cinnamal, Carmine, และสารอื่นๆอีกรวม 9 ชนิด
ด้วยความแม่นยำ 2.20 % (มีข้อมูลสาร 106 ชนิด, จาก 4813 ชนิด)
หมายเหตุ: ยิ่งป้อนข้อมูลประวัติการใช้ผลิตภัณฑ์บำรุงผิวของคุณให้มากขึ้น ยิ่งทำให้ความแม่นยำในการคาดเดามากขึ้น

!!เราขอแนะนำ!! : Urban Decay Rehab Makeup Prep Lip Love
ประเภท: 'Face Skincare', 'Lip Care', 'Lip Scrubs'
มีความคล้ายผลิตภัณฑ์เดิม 46.19 %
!หรืออันดับ! 2 :  Kiss My Face Organic Ginger Mango Lip Balm 
ประเภท: 'Lip Balms', 'Face Skincare', 'Lip Care'
มีความคล้ายผลิตภัณฑ์เดิม 39.53 %
!หรืออันดับ! 3 :  Kiss My Face Organic Cranberry Orange Lip Balm 
ประเภท: 'Lip Balms', 'Face Skincare', 'Lip Care'
มีความคล้ายผลิตภัณฑ์เดิม 37.27 %
----------------------------------------------------------------




In [20]:
df_customer

,name,rmd,alg_ind,nalg_ind,alg_ing,nalg_ing,cosine
0,mean,{20740: 8281},[20740],"[1789, 281, 197, 9153, 4402, 3018, 24653]","{'Eugenol': [7, None, ['Denaturant', 'Fragranc...","{'Water': [1, None, ['Solvent']], 'Butyrosperm...","[{0.043852900965351466: 25124, 0.2236067977499..."
1,may,"{3171: 19687, 18197: 12196, 3300: 12196, 12559...","[3171, 18197, 3300, 12559, 22615, 3267]","[3832, 1973, 879, 15281, 13618, 10387, 1734, 8...",{'Hamamelis Virginiana (Witch Hazel) Extract':...,"{'Rehmannia Glutinosa Root Extract': [1, None,...","[{0.1283881477532739: 28853, 0.206196524710580..."
2,oly,{8306: 8779},[8306],"[580, 3593, 3308, 18067, 19519]","{'Pyrus Malus (Apple) Fruit Extract': [1, None...","{'Water': [1, None, ['Solvent']], 'Hydrolyzed ...","[{0.11322770341445959: 29477, 0.09622504486493..."
3,preaw-1,{3267: 17739},[3267],"[27910, 23525]","{'C13-14 Isoparaffin': [1, 'B', ['Emollient', ...","{'Ethylparaben': [4, 'A', ['Fragrance', 'Prese...","[{0.1286978904175574: 23592, 0.116642368703960..."
4,preaw-2,{3267: 19725},[3267],[3333],"{'Titanium Dioxide': [3, None, ['Cosmetic Colo...","{'Ethylparaben': [4, 'A', ['Fragrance', 'Prese...","[{0.06734350297014739: 22037, 0.11461365101225..."
5,cinth,{3028: 16311},[3028],"[3220, 4451]",{'Hydroxyethyl Acrylate/Sodium Acryloyldimethy...,"{'Water': [1, None, ['Solvent']], 'Butyrosperm...","[{0.05976143046671968: 28072, 0.0: 29749, 0.05..."


In [21]:
#df_customer.to_csv('customer.csv', index = False, header=True)